In [20]:
import pandas as pd
df = pd.read_csv('Tweets.csv')
df['Date'] = df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

In [16]:
len(df[df['Date'] > '2022-03-14'])

1870

In [24]:
len(df[df['Date'].dt.strftime('%Y-%m-%d') == '2022-03-16'])

158

In [33]:
from datetime import date, timedelta

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2022, 3, 14)
end_date = date(2022, 3, 30)
for single_date in daterange(start_date, end_date):
    print(single_date,":",len(df[df['Date'].dt.strftime('%Y-%m-%d') == str(single_date)]))

AttributeError: 'Series' object has no attribute 'date'

In [27]:
len(df)

8224

In [32]:
df['Date'][0].date()

datetime.date(2022, 3, 29)